# Welcome to Quick Start.

### This notebook tests the microservice, provides insight on how well it performs and showcases how to use the microservice using python.

**Note: You should have the model downloaded for this notebook to work. You can refer to the repository's readme file on the steps to follow before starting testing here. If you're all set let's get started:**



### Imports

In [ ]:
import matplotlib.pyplot as plt
import cv2
from datasets import load_dataset
import random
import numpy as np
import time
import requests
from PIL import Image
from io import BytesIO
import base64
import urllib.parse

## Download Datasets for Testing

If you don't know where to start in testing the dataset, you can run some of the cells below to download datasets.

### Traffic Dataset

In [ ]:
# Define the dataset name
traffic_dataset_name = "Francesco/road-traffic"

# Load the dataset
traffic_dataset = load_dataset(traffic_dataset_name)
traffic_dataset = traffic_dataset['train']

print("Dataset downloaded successfully!")

### House Rooms Image Dataset 

In [ ]:
# Define the dataset name
house_dataset_name = "xyfang/indoor_rooms"

# Load the dataset
house_dataset = load_dataset(house_dataset_name)
house_dataset = house_dataset["train"]

print("Dataset downloaded successfully!")

## Build the Docker Image

Run the following command on the directory, more instructions can be found on the documents.

docker-compose up

**You must leave it running while you test** 

Afterwards you can press ctrl+c to stop running and close the terminal page when it stops.

## Test the microservice

The output of the microservice is a JSON file. Inside the JSON file, the output image is encoded as a base64 string. For showcasing purposes, we can decode the base64 string and display it as an output image ourselves. It's important to note that this decoding process is not performed by the microservice.

The next cell contains some required methods and variables. These methods and variables are essential for the functionality of the code in this Jupyter Notebook. 

In [ ]:
url = 'http://localhost:5001/detect'

#write code that selects 50 random images from the datasets, shows the original and the one outputted from the request, waits 10 seconds, moves on to the next one for 50 images.
random.seed(111)
traffic_indices = random.sample(range(len(traffic_dataset)), 5)
house_indices = random.sample(range(len(house_dataset)), 5)

def url_label(label):
    encoded_label = urllib.parse.quote(label)
    labeled_url = url + "/" + encoded_label
    return labeled_url

# Function to convert image to JPEG bytes
def convert_to_jpeg_bytes(image_array):
    image = Image.fromarray(image_array)
    with io.BytesIO() as output:
        image.save(output, format="JPEG")
        jpeg_image = output.getvalue()
    return jpeg_image

### Requests without label filtering

Next cell shows randomly selected images being run through our microservice without label filtering, watch the images roll as the microservice processes them:


In [ ]:
for idx in traffic_indices:
    data = traffic_dataset[idx]
    image = np.array(data['image'])
    image = convert_to_jpeg_bytes(image)

    # Convert the image data to a BytesIO object
    image_io = BytesIO(image)
    pil_image = Image.open(image_io)

    # Run the image through the YOLO model
    files = {'image': ('image.jpg', image, 'image/jpeg')}
    
    # Send the POST request
    response = requests.post(url, files=files)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the JSON response
        print(response.json())
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

    # Convert the output image 
    output_image = BytesIO(base64.b64decode(response.json()['image']))
    pil_output_image = Image.open(output_image)
    
    # Display the original and output images side by side
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(pil_image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")
    ax[1].imshow(pil_output_image)
    ax[1].set_title("Output Image")
    ax[1].axis("off")
    plt.show()
    time.sleep(1)


for idx in house_indices:
    data = house_dataset[idx]
    image = data['image']
    image = np.array(data['image'])
    image = convert_to_jpeg_bytes(image)
    
    image_io = BytesIO(image)
    pil_image = Image.open(image_io)

    # Run the image through the YOLO model
    files = {'image': image}
    
    # Send the POST request
    response = requests.post(url, files=files)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the JSON response
        print(response.json())
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

        
    # Convert the output image 
    output_image = BytesIO(base64.b64decode(response.json()['image']))
    pil_output_image = Image.open(output_image)
    
    # Display the original and output images side by side
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(pil_image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")
    ax[1].imshow(pil_output_image)
    ax[1].set_title("Output Image")
    ax[1].axis("off")
    plt.show()
    time.sleep(1)

### Requests with label filtering

The next cell shows how to use label filtering, the key point here is the special url generated for the filters. The url gets the addition /label. See the results by running the cell:

In [ ]:
def request_with_label_filtering(data, label):
    image = data['image']
    image = np.array(data['image'])
    image = convert_to_jpeg_bytes(image)
    labeled_url = url_label(label)
    image_io = BytesIO(image)
    pil_image = Image.open(image_io)

    # Run the image through the YOLO model
    files = {'image': image}
    
    # Send the POST request
    response_no_filter = requests.post(url, files=files)

    # Check if the request was successful
    if response_no_filter.status_code == 200:
        # Print the JSON response
        print(response_no_filter.json())
    else:
        print(f"Request failed with status code: {response_no_filter.status_code}")
        print(response_no_filter.text)

    # Convert the output image 
    output_image_nf = BytesIO(base64.b64decode(response_no_filter.json()['image']))
    pil_output_image_nf = Image.open(output_image_nf)

    # Send the POST request for label filtering
    response_with_filter = requests.post(labeled_url, files=files)

    # Check if the request was successful
    if response_with_filter.status_code == 200:
        # Print the JSON response
        print(response_with_filter.json())
    else:
        print(f"Request failed with status code: {response_with_filter.status_code}")
        print(response_with_filter.text)

    # Convert the output image 
    output_image_wf = BytesIO(base64.b64decode(response_with_filter.json()['image']))
    pil_output_image_wf = Image.open(output_image_wf)
    
    # Display the original and output images side by side
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(pil_image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")
    ax[1].imshow(pil_output_image_nf)
    ax[1].set_title("Output Image (No Filter)")
    ax[1].axis("off")
    ax[2].imshow(pil_output_image_wf)
    ax[2].set_title(f"Output Image (With {label} Filter)")
    ax[2].axis("off")
    plt.show()
    time.sleep(1)

request_with_label_filtering(traffic_dataset[traffic_indices[2]], "Truck")
request_with_label_filtering(traffic_dataset[traffic_indices[2]], "Car")
request_with_label_filtering(traffic_dataset[traffic_indices[2]], "Traffic Light")

request_with_label_filtering(house_dataset[house_indices[4]], "Sofa")
request_with_label_filtering(house_dataset[house_indices[4]], "Chair")
request_with_label_filtering(house_dataset[house_indices[4]], "Book")

## Conclusion and how to explore the microservice:

As you can see the microservice performs well and the label filtering option is a nice feature for anyone looking to focus on one type of object.

You can test the microservice by Python http requests or use an app like Postman. More info on how to explore the microservice deeply can be found on the readme file.